### Import settings

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.11:2.3.0 pyspark-shell'

In [ ]:
import findspark
findspark.init('/opt/spark')
from pyspark import SparkContext,SparkConf
sc = SparkContext()
#with Cluster
#conf = (SparkConf()
#         .setMaster("spark://10.200.5.39:7077")
#         .set("spark.driver.host","10.200.5.39") 
#         .set("spark.executor.memory","58g")
#         .set('spark.driver.memory', '60G')
#         .setAppName("readApp"))
#sc = SparkContext(conf=conf)

In [ ]:
import pyspark
import binascii
from pyspark.sql import SQLContext
from functools import reduce
import pygraphviz
import pyspark.sql.functions as f
from IPython.display import Image
from networkx.drawing.nx_pydot import write_dot
sqlContext = SQLContext(sc)

### Functions space

In [ ]:
path="/Kriptosare.class/analysis/bitcoin/"

In [ ]:
# Load from Cassandra
def load_and_get_table_df(keys_space_name, table_name):
    table_df = sqlContext.read\
        .format("org.apache.spark.sql.cassandra")\
        .options(table=table_name, keyspace=keys_space_name)\
        .load()
    return table_df

# Convert bytearray to str
def using_str_format(test_obj) -> str:
    return "".join("{:02x}".format(x) for x in test_obj)

# Retrive a user in the output address dataframe
def user_find(x):
    return df_output_addresses_tag_grpby_addr.where(f.col("address")==str(x)).select("user")



### Import data, read from cassandra

In [ ]:
# Load raw data
df_blocks_original = load_and_get_table_df("blockchain_data", "block")

In [ ]:
df_transactions_original = load_and_get_table_df("blockchain_data", "transaction")

In [ ]:
#split_col = f.split(df_blocks.select('txs'), ',')
df_blocks = df_blocks_original.withColumn('txs',f.explode(f.col('txs')))
df_blocks = df_blocks.withColumn('txs',f.hex(f.col('txs')))
df_blocks = df_blocks.withColumn('block_hash',f.hex(f.col('block_hash')))

In [ ]:
# Trasform transaction dataframe in order to separate different value in VOUT field
df_transactions = df_transactions_original.withColumn('tx_hash',f.hex(f.col('tx_hash')))
df_transactions = df_transactions.withColumn('tx_id',f.hex(f.col('tx_id')))
df_transactions = df_transactions.withColumn("vout", f.explode(f.col('vout')))

df_transactions = df_transactions.withColumn("amount",df_transactions['vout']['value']) 
df_transactions = df_transactions.withColumn("vout_idx",df_transactions['vout']['n']) 
df_transactions = df_transactions.withColumn("address",f.explode(df_transactions['vout']['addresses'])) 
df_transactions = df_transactions.drop("vout")

df_transactions = df_transactions.withColumn("vin", f.explode(f.col('vin')))
df_transactions = df_transactions.withColumn("vin_txid",f.hex(df_transactions['vin']['txid'])) 
df_transactions = df_transactions.withColumn("vin_vout",df_transactions['vin']['vout']) 
df_transactions = df_transactions.drop("vin")

In [ ]:
for i in range(0,10):
    df_transactions_original_part2 = df_transactions.filter((f.col("height")>=(i*10000)&(f.col("height")<(i+1)*10000)))
    df_transactions_original_part2.write\
    .format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("codec", "org.apache.hadoop.io.compress.GzipCodec")\
    .save(path+"transaction_part"+str(i))